# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 15 2022 10:46AM,239646,15,VT80173885,"Virtus Pharmaceuticals, LLC",Released
1,Jun 15 2022 10:46AM,239645,15,8145359,"Mizner Bioscience, LLC",Released
2,Jun 15 2022 10:46AM,239645,15,8145360,"Mizner Bioscience, LLC",Released
3,Jun 15 2022 10:42AM,239644,15,8147386,"Carwin Pharmaceutical Associates, LLC",Released
4,Jun 15 2022 10:42AM,239644,15,8147387,"Carwin Pharmaceutical Associates, LLC",Released
5,Jun 15 2022 10:42AM,239644,15,8147388,"Carwin Pharmaceutical Associates, LLC",Released
6,Jun 15 2022 10:42AM,239644,15,8147389,"Carwin Pharmaceutical Associates, LLC",Released
7,Jun 15 2022 10:42AM,239644,15,8147390,"Carwin Pharmaceutical Associates, LLC",Released
8,Jun 15 2022 10:40AM,239641,20,ALUR337689236,Alumier Labs Inc.,Released
9,Jun 15 2022 10:40AM,239641,20,ALUR151325240,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
48,239642,Released,1
49,239643,Released,1
50,239644,Released,5
51,239645,Released,2
52,239646,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239642,NaN,NaN,1.0
239643,NaN,NaN,1.0
239644,NaN,NaN,5.0
239645,NaN,NaN,2.0
239646,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239446,74.0,0.0,79.0
239551,6.0,20.0,2.0
239554,8.0,1.0,0.0
239555,0.0,0.0,3.0
239556,0.0,0.0,3.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239446,74,0,79
239551,6,20,2
239554,8,1,0
239555,0,0,3
239556,0,0,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239446,74,0,79
1,239551,6,20,2
2,239554,8,1,0
3,239555,0,0,3
4,239556,0,0,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239446,74,,79
1,239551,6,20,2
2,239554,8,1,
3,239555,,,3
4,239556,,,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 15 2022 10:46AM,239646,15,"Virtus Pharmaceuticals, LLC"
1,Jun 15 2022 10:46AM,239645,15,"Mizner Bioscience, LLC"
3,Jun 15 2022 10:42AM,239644,15,"Carwin Pharmaceutical Associates, LLC"
8,Jun 15 2022 10:40AM,239641,20,Alumier Labs Inc.
34,Jun 15 2022 10:39AM,239643,10,Template KITS
35,Jun 15 2022 10:36AM,239642,18,"ClearSpec, LLC"
36,Jun 15 2022 10:32AM,239640,10,Emerginnova
38,Jun 15 2022 10:31AM,239636,15,"Alliance Pharma, Inc."
63,Jun 15 2022 10:29AM,239630,10,ISDIN Corporation
128,Jun 15 2022 10:28AM,239637,12,Hush Hush


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 15 2022 10:46AM,239646,15,"Virtus Pharmaceuticals, LLC",,,1
1,Jun 15 2022 10:46AM,239645,15,"Mizner Bioscience, LLC",,,2
2,Jun 15 2022 10:42AM,239644,15,"Carwin Pharmaceutical Associates, LLC",,,5
3,Jun 15 2022 10:40AM,239641,20,Alumier Labs Inc.,,,26
4,Jun 15 2022 10:39AM,239643,10,Template KITS,,,1
5,Jun 15 2022 10:36AM,239642,18,"ClearSpec, LLC",,,1
6,Jun 15 2022 10:32AM,239640,10,Emerginnova,,,2
7,Jun 15 2022 10:31AM,239636,15,"Alliance Pharma, Inc.",,,25
8,Jun 15 2022 10:29AM,239630,10,ISDIN Corporation,,,65
9,Jun 15 2022 10:28AM,239637,12,Hush Hush,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 15 2022 10:46AM,239646,15,"Virtus Pharmaceuticals, LLC",1,,
1,Jun 15 2022 10:46AM,239645,15,"Mizner Bioscience, LLC",2,,
2,Jun 15 2022 10:42AM,239644,15,"Carwin Pharmaceutical Associates, LLC",5,,
3,Jun 15 2022 10:40AM,239641,20,Alumier Labs Inc.,26,,
4,Jun 15 2022 10:39AM,239643,10,Template KITS,1,,
5,Jun 15 2022 10:36AM,239642,18,"ClearSpec, LLC",1,,
6,Jun 15 2022 10:32AM,239640,10,Emerginnova,2,,
7,Jun 15 2022 10:31AM,239636,15,"Alliance Pharma, Inc.",25,,
8,Jun 15 2022 10:29AM,239630,10,ISDIN Corporation,65,,
9,Jun 15 2022 10:28AM,239637,12,Hush Hush,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 15 2022 10:46AM,239646,15,"Virtus Pharmaceuticals, LLC",1,,
1,Jun 15 2022 10:46AM,239645,15,"Mizner Bioscience, LLC",2,,
2,Jun 15 2022 10:42AM,239644,15,"Carwin Pharmaceutical Associates, LLC",5,,
3,Jun 15 2022 10:40AM,239641,20,Alumier Labs Inc.,26,,
4,Jun 15 2022 10:39AM,239643,10,Template KITS,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 15 2022 10:46AM,239646,15,"Virtus Pharmaceuticals, LLC",1.0,NaN,NaN
1,Jun 15 2022 10:46AM,239645,15,"Mizner Bioscience, LLC",2.0,NaN,NaN
2,Jun 15 2022 10:42AM,239644,15,"Carwin Pharmaceutical Associates, LLC",5.0,NaN,NaN
3,Jun 15 2022 10:40AM,239641,20,Alumier Labs Inc.,26.0,NaN,NaN
4,Jun 15 2022 10:39AM,239643,10,Template KITS,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 15 2022 10:46AM,239646,15,"Virtus Pharmaceuticals, LLC",1.0,0.0,0.0
1,Jun 15 2022 10:46AM,239645,15,"Mizner Bioscience, LLC",2.0,0.0,0.0
2,Jun 15 2022 10:42AM,239644,15,"Carwin Pharmaceutical Associates, LLC",5.0,0.0,0.0
3,Jun 15 2022 10:40AM,239641,20,Alumier Labs Inc.,26.0,0.0,0.0
4,Jun 15 2022 10:39AM,239643,10,Template KITS,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2875188,146.0,5.0,8.0
12,718881,4.0,0.0,0.0
15,1198017,112.0,0.0,74.0
18,1916756,7.0,1.0,0.0
19,2635606,32.0,9.0,5.0
20,718811,29.0,20.0,6.0
21,718777,2.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2875188,146.0,5.0,8.0
1,12,718881,4.0,0.0,0.0
2,15,1198017,112.0,0.0,74.0
3,18,1916756,7.0,1.0,0.0
4,19,2635606,32.0,9.0,5.0
5,20,718811,29.0,20.0,6.0
6,21,718777,2.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,146.0,5.0,8.0
1,12,4.0,0.0,0.0
2,15,112.0,0.0,74.0
3,18,7.0,1.0,0.0
4,19,32.0,9.0,5.0
5,20,29.0,20.0,6.0
6,21,2.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,146.0
1,12,Released,4.0
2,15,Released,112.0
3,18,Released,7.0
4,19,Released,32.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,18,19,20,21
Status,,,,,,,
Completed,8.0,0.0,74.0,0.0,5.0,6.0,0.0
Executing,5.0,0.0,0.0,1.0,9.0,20.0,1.0
Released,146.0,4.0,112.0,7.0,32.0,29.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,18,19,20,21
0,Completed,8.0,0.0,74.0,0.0,5.0,6.0,0.0
1,Executing,5.0,0.0,0.0,1.0,9.0,20.0,1.0
2,Released,146.0,4.0,112.0,7.0,32.0,29.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,18,19,20,21
0,Completed,8.0,0.0,74.0,0.0,5.0,6.0,0.0
1,Executing,5.0,0.0,0.0,1.0,9.0,20.0,1.0
2,Released,146.0,4.0,112.0,7.0,32.0,29.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()